### This notebook makes an effort to determine exactly what subjects, and what input datastream was used in https://www.biorxiv.org/content/10.1101/2020.05.08.084707v1

In [1]:
from glob import glob 
import pandas as pd
import csv
import numpy as np
import chardet
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

We analyzed data from 7,321 of the 11,878 participants. This subset resulted from the
following exclusion, which attempted to eliminate incomplete data or data that were already
flagged as problematic by the ABCD study organizers. First, we attempted to download 8,811
participants from the “FastTrack Recommended Active Series” from the NIMH Data Archive,
which has gone through some quality assurance for the associated imaging files by the ABCD
study organizers. Of these, 8,776 files were successfully downloaded, but a subset did not
include stop-signal data, leaving ​ 7,906 subjects. Of these, only 7,347 included ​ summary scores
from the Stop Signal Task in the ABCD Data Release 2.0. Finally, 26 subjects were removed who did not have two complete runs with 180 trials each, leaving us with a total of 7,321
complete datasets.

Claim 1: First, we attempted to download 8,811 participants from the “FastTrack Recommended Active Series” from the NIMH Data Archive

As of 5/13/2020, the recommended active series (for all modalities) has 10709 subjects avaliable, and alternate values based on when extra subjects were added would be 10404 before oct 2019, 9315 before sept 2019, and 8151 before aug 2019.

Alternatively, they could have tried to select just the fMRI fast track reccomended data:
This starts with 10390 subejcts avaliable as of 5/13/2020, goes down to 10098 as of oct 2019, down to 9032 as of sep 2019, and then down to 7904 as of oct 2019.

Lastly, they could have tried to select just the SST data:
As of may 2020 there are 9181 subjects avaliable, and assuming downloaded before oct 2019, there are 8279 subjects avaliable.

In summary, it is unclear what 8,811 participants cam from - but lets assume they did as they said and downloading the full "FastTrack Recommended Active Series"

From https://github.com/mckenziephagen/ABCD_Stop_Signal/blob/eaf3c8d500971ba8c458c59f743fd3040eeb1133/scripts/General/s3_keys.ipynb

We will try to re-create what they did to download the data from this above deleted script.

In [2]:
def load_img_manifest(name):
    
    # This is just where my AWS downloader saved the manifests, and to keep things
    # simple I would just rename them + keep them in the same folder
    df = pd.read_csv('/home/sage/AWS_downloads/' + name, sep='\t', error_bad_lines=False,
                     engine = 'python', quoting=csv.QUOTE_NONE,
                     header=[0]).replace('"','', regex=True)
    df = df.drop(df.index[0], axis=0)
    return df

Specifically what is done below is that I just try downloading the image manifest for a number of different permutations of possible reccomended active series fasttrack data from nda. I.e., 'all' is if you download all of the fasttrack data, for all imaging data, all_minus1 would be downloading the same but without the very last added data source, minus 2 would be without the last 2, 'frmi' would be if you downloading just the reccomended active series frmi datam etc..., and 'fast' is downloading all the subjects from the just fasttrack data (no reccomended active series flag), and on and on. The details don't reallyt matter as you'll see below.

In [3]:
df1 = load_img_manifest('all.txt')
df2 = load_img_manifest('all_minus1.txt')
df3 = load_img_manifest('all_minus2.txt')
df4 = load_img_manifest('all_minus3.txt')
df5 = load_img_manifest('fmri.txt')
df6 = load_img_manifest('just_sst.txt')
df7 = load_img_manifest('fast.txt')
df8 = load_img_manifest('fast_minus1.txt')
df9 = load_img_manifest('fast_minus2.txt')
df10 = load_img_manifest('null.txt')

In [4]:
all_dfs = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10]

In [5]:
for df in all_dfs:
    print(len(df), len(df['"subjectkey"'].unique()))

301838 10709
296738 10673
279281 10651
260770 10637
119443 10390
22489 9181
360099 11769
351299 11767
332633 11763
301838 10709


None match the length, 250331, as seen in the s3_keys script... Interesting ? And we still dont match the number of subjects

Rest of steps of steps from original analysis

image_descriptions = ['ABCD-SST-fMRI']

s3_df = pd.DataFrame() 
for task in image_descriptions: 
    s3_df = s3_df.append(sub_df[sub_df.iloc[:, 11] == task])
    
s3_df = s3_df[s3_df['"visit"'] == 'baseline_year_1_arm_1']

with open('all_SST_s3s.txt', 'w') as file:
    for i in s3_df['"image_file"']: 
        file.write(i  + "\n")

Convert the above to just: filten by task then eventname

In [6]:
for df in all_dfs:
    df = df[df['"image_description"'] == 'ABCD-SST-fMRI']
    df = df[df['"visit"'] == 'baseline_year_1_arm_1'] 
    
    print(df.shape, len(df['"subjectkey"'].unique()))

(17243, 112) 8811
(17243, 112) 8811
(17176, 112) 8777
(16854, 112) 8618
(17243, 112) 8811
(17243, 112) 8811
(20296, 112) 10166
(20296, 112) 10166
(20220, 112) 10131
(17243, 37) 8811


At last 8811! We have have where they got the number! Next, download the data as they did.

In [7]:
# Lets save our keys to download as the 1st one, which I'm confident now is the one they used
s3_df = all_dfs[0]

with open('all_SST_s3s.txt', 'w') as file:
    for i in s3_df['"image_file"']: 
        file.write(i  + "\n")

Specifically, after these keys are saved, the nda_tools download script is used to perform the download

the command: downloadcmd all_SST_s3s.txt -t
is run.

Then to extract the output the following script is submitted to the UVM VACC cluster - which is designed to be a parrellizable to an arbitrary number of jobs script that extracts the event file from downloaded raw fast track data, and then deleted the original tar file

In [ ]:
import tarfile, os, random

def extract_event(file, dr):

    tar = tarfile.open(file, "r:gz")
    for m in tar.getmembers()[::-1]:

        if 'EventRelatedInformation' in m.name:

            name = m.name
            new_dr = os.path.join(dr, '/'.join(name.split('/')[:-1]))
            os.makedirs(new_dr, exist_ok=True)

            file_loc = os.path.join(dr, name)
            event_info = tar.extract(m, dr)

            os.remove(file)
            break

dr = '/users/s/a/sahahn/SST_Data'
download_dr = '/users/s/a/sahahn/AWS_downloads'

folders = os.listdir(download_dr)
folders = [os.path.join(download_dr, folder) for folder in folders]
random.shuffle(folders)

for folder in folders:
    files = os.listdir(folder)
    files = [os.path.join(folder, file) for file in files]

    files = [file for file in files if file.endswith('.tgz')]
    random.shuffle(files)

    for file in files:
        if file in [os.path.join(folder, f) for f in os.listdir(folder)]:
            os.rename(file, file + '-extracting')
            extract_event(file + '-extracting', dr)

## Part 2 - creating the merged / concated event file

The next step in tracking down the number of subjects comes from:

"Of these, 8,776 files were successfully downloaded, but a subset did not
include stop-signal data, leaving ​ 7,906 subjects. Of these, only 7,347 included ​ summary scores
from the Stop Signal Task in the ABCD Data Release 2.0. Finally, 26 subjects were removed who did not have two complete runs with 180 trials each, leaving us with a total of 7,321
complete datasets.", we will also reference https://github.com/mckenziephagen/ABCD_Stop_Signal/blob/eaf3c8d500971ba8c458c59f743fd3040eeb1133/scripts/SST_manuscript/concat.ipynb
In this notebook, which is another deleted notebook.



## Issue 1

What I did was tried to download the same set of files they did. In my download, 8809, files were both successfully downloaded + have the event information file (just SST data). So, apparently 33 subjects in their download just didn't work? I mean, some didn't work in my download, but 33 is a bit of a different (granted the caveat here is that I only kept the event file info, maybe they had already filtered for missing stop-signal data?? 



I further put the sst event files in their original directory structure w/ even the saming naming as seen in the above notebook, SST_Data, for consistency w/ their work.

In [8]:
from glob import glob 
import pandas as pd
import csv
import numpy as np
import chardet
import os

In [9]:
# This file globbing is the one they may have used based on the shared script
subs = list(set([i.split('-')[1].split('/')[0]
                 for i in glob('data/SST_Data/sub-*/ses-baselineYear1Arm1/func/*SST*.txt')]))
len(subs)

8146

In [10]:
main_dr = 'data/SST_Data/'
subjects = os.listdir(main_dr)
print(len(set(subjects)))

8808


Why is this number different from the number of subjects in the directory?

A little investigation reveals this is due to some are saved as .csv, lets look at it more closely

In [11]:
n_mixed = 0

for subject in subjects:
    sub_dr = os.path.join(main_dr, subject)
    sessions = os.listdir(sub_dr)
    
    if 'ses-baselineYear1Arm1' not in sessions:
        print(sessions)
        
    else:
        session_dr = os.path.join(sub_dr, 'ses-baselineYear1Arm1')
        f_files = os.listdir(session_dr)
        
        if 'func' not in f_files:
            print(f_files)

        else:
            f_dr = os.path.join(session_dr, 'func')
            files = os.listdir(f_dr)
            if 'SST' not in files:
                for file in files:
                    if '.txt' not in file or 'SST' not in file:
                        print(file)
                        
                # How many cases where 1 txt 1 csv
                num_txt = len([f for f in files if '.csv' in f])
                num_csv = len([f for f in files if '.txt' in f])
                
                if num_txt == 1 and num_csv == 1:
                    n_mixed += 1

ABCD-SST-fMRI_run-20170616154139-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20170616154824-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180625103553-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180625102736-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20181008162852-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20181008162255-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180523174759-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180523175418-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180726151456-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180726152144-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20161119105057-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20161119105833-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20160926160659-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20160926161429-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180825120007-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180825120631-EventRelatedInformation.csv
ABCD-SST-fMRI_run-201611

ABCD-SST-fMRI_run-20161114120338-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20161114120959-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180927183755-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180927183148-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20171020141446-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180808152001-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180808152624-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180728113613-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180728112938-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180731141405-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20180731140748-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20181006105539-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20181006110204-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20170307161512-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20170307162227-EventRelatedInformation.csv
ABCD-SST-fMRI_run-20161222181348-EventRelatedInformation.csv
ABCD-SST-fMRI_run-201612

In [12]:
# So this is the number of subjects with 1 .txt eventfile and .csv
n_mixed

10

In [13]:
missed_subs = list(set([i.split('-')[1].split('/')[0]
                   for i in glob('data/SST_Data/sub-*/ses-baselineYear1Arm1/func/*SST*.csv')]))
len(missed_subs)

683

## Issue 2

a number of subjects were systematically excluded before their event related information was saved as a csv... with another 10 cases where it appeared there was only 1 valid run, but only because their was one event info .csv and one .txt

In [14]:
event_files = glob('data/SST_Data/sub-*/ses-baselineYear1Arm1/func/*SST*.txt')
missed_event_files = glob('data/SST_Data/sub-*/ses-baselineYear1Arm1/func/*SST*.csv')
len(event_files), len(missed_event_files)

(15922, 1314)

In [15]:
# I add this function, because eventfiles are duplicated in the fast track data
def remove_duplicates(event_files):
    
    subjs = np.array([e.split('/')[2] for e in event_files])
    to_delete = []

    for s in subjs:
        inds = np.where(subjs == s)[0]
        if len(inds) > 1:
            for i in inds[1:]:
                to_delete.append(i)
                
    to_delete = list(set(to_delete))
    event_files = np.delete(np.array(event_files), to_delete)
    
    return list(event_files)

In [16]:
# Remove duplicates, as see there are ~twice as many
event_files = remove_duplicates(event_files)
missed_event_files = remove_duplicates(missed_event_files)

len(event_files), len(missed_event_files)

(8146, 683)

In [17]:
def get_encoding(file): 
    """
    retrieves character encoding of file to be passed to 
    pd.read_csv()
    """
    with open(file, mode = "rb") as f: 
        rawdata = f.read()
        return chardet.detect(rawdata)['encoding']

The below chunk of code is slightly modified from their shared code on this merging, I just added a try expect block to catch failed files, and a check to add 'weird' files. Extremely percise language, I know

In [18]:
i=0

dfp = []
failed_files = []
weird_files = []

for file in event_files: 
    subj_id = file.split('/')[2].split('-')[1].replace('NDAR', 'NDAR_')
    encoding = get_encoding(file)
    
    # In a number of cases, the original code fails a number of times, lets' save these files
    # to look at more closely
    try:
        
        df = pd.read_csv(file, sep = '\t', encoding = encoding)
        
        if df.columns[0] != 'ExperimentName': #to fix header differences
            df = pd.read_csv(file, sep = '\t', skiprows=[0], encoding = encoding)
            
        df['src_subject_id'] = subj_id
        
        if 'Go.RESP' not in list(df):
            weird_files.append(file)
        else:
            dfp.append(df)
    
    except:
        failed_files.append(file)
        
    i+=1

In [19]:
event_df = pd.concat(dfp, sort=True, axis=0)

for col in event_df.columns:
    if 'Unnamed' in col or 'task' in col or 'Test' in col or 'Wait' in col:
        del event_df[col]
        
del dfp # Save memory   

event_df.to_csv('merged_data/full_SST_concat.csv')

Notably some of the weird and bad files might be salvagable... likewise we have still been just ignoring all the csvs.

The provided loading code (caveat: not officially provided, s.t., the actual final version could have done something different, i.e., the referenced code only goes through this process on 100 event files, s.t., when they actually ran this on all the subjects they very likely encountered a simmilar error - but who knows if they fixed it or not) fails for 26 files

In [20]:
## My artesinal take on creating a loading function robust to the weird errors

def load_event_file(file, sep='\t'):
    subj_id = file.split('/')[2].split('-')[1].replace('NDAR', 'NDAR_')
    
    encoding = get_encoding(file)
    try:
        df = pd.read_csv(file, sep =sep, encoding = encoding)
    except:
        try:
            df = pd.read_csv(file, sep =sep, encoding = encoding, skiprows=[0, 1])
        except:
            return None

    
    if df.columns[0] == '0-ExperimentName':
        return None
    
    if df.columns[0] != 'ExperimentName':
        df = pd.read_csv(file, sep=sep, skiprows=[0], encoding = encoding)
        
    if '"ExperimentName' in df.columns:
        df = df.rename({'"ExperimentName': 'ExperimentName'}, axis=1)
        
    if df.columns[0] == 'EXPNAME':
        df = pd.read_csv(file, sep=sep, skiprows=[0, 1, 2], encoding = encoding)
        
    try:
        if 'SST' not in df['ExperimentName'][0]:
            return None
    except TypeError:
        return None
    
    df['src_subject_id'] = subj_id
        
    return df

In [21]:
bad_files = weird_files + failed_files
len(bad_files)

23

In [22]:
extra_dfs = []
for file in bad_files:
    df = load_event_file(file)
    
    if df is not None:
        extra_dfs.append(df)

In [23]:
## Also load the missed event files, w/ more hacky tricks to try and handle all of the weird errors

missed_df = []

for file in missed_event_files:
    
    try:
        df = load_event_file(file, sep=',')
    except KeyError:
        
        try:
            df = load_event_file(file)
        except KeyError:
            
            try:
                df = load_event_file(file, sep='\\t')
            except:
                df = None
    
    if df is not None:
        missed_df.append(df)

/home/sage/anaconda3/envs/home/lib/python3.7/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/sage/anaconda3/envs/home/lib/python3.7/site-packages/ipykernel_launcher.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [24]:
len(extra_dfs), len(missed_df)

(11, 681)

Still missing some... but some of the failed files especially are either actually messed up, or in a few cases contain data from a different task. Oh well, stressing about the extact number of subjects is not all that productive.

In [25]:
extra_df = pd.concat(extra_dfs, sort=True, axis=0)
extra_df.to_csv('merged_data/extra_concat.csv')

missed_df = pd.concat(missed_df, sort=True, axis=0)
missed_df.to_csv('merged_data/missed_concat.csv')

Lastly, lets do two things, prepare some lower memory versions (i.e., we can get rid of columns which are never used in the remaining analysis), and also we can create all_concat.csv, which represents a merged version of all avaliable data, i.e., the full_SST_concat (my best guess at the subjects they used), with the extra_concat and missed_concat.

In [26]:
raw_concat = pd.read_csv('merged_data/full_SST_concat.csv', low_memory=False)

used_cols = ['Procedure[SubTrial]', 'Procedure[Trial]', 'Go.RESP', 'Go.CRESP', 'Fix.RESP', 
             'StopSignal.RESP', 'SSD.RESP', 'SSD.RT', 'Stimulus',
             'Go.RT', 'Go.Duration', 'Fix.RT', 'StopSignal.RT', 'StopSignal.Duration',
             'SSDDur', 'NARGUID', 'src_subject_id', 'TrialCode']

raw_concat = raw_concat[used_cols]
raw_concat.to_csv('merged_data/SST_concat.csv')

extra_concat = pd.read_csv('merged_data/extra_concat.csv', low_memory=False)[used_cols]
missed_concat = pd.read_csv('merged_data/missed_concat.csv', low_memory=False)[used_cols]

all_concat =  pd.concat([raw_concat, extra_concat, missed_concat], axis=0)
all_concat.to_csv('merged_data/all_concat.csv')